1. hyperparemter sweep
2. Comparethe result  of the runs in the MLflow  UI
3. Choose best run and register model
4. Deploy the model on Rest API
5. Make a container and upload to cloud

In [1]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

In [2]:
### load the dataset
data=pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [3]:
### split the data into train, validation and test set

train,test=train_test_split(data,test_size=0.2)
train

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
2399,6.1,0.16,0.24,1.4,0.046,17.0,77.0,0.99319,3.66,0.57,10.300000,6
1604,6.7,0.41,0.01,2.8,0.048,39.0,137.0,0.99420,3.24,0.35,9.500000,5
2818,6.0,0.11,0.47,10.6,0.052,69.0,148.0,0.99580,2.91,0.34,9.300000,4
2892,6.7,0.42,0.46,9.7,0.054,67.0,234.0,0.99848,3.23,0.50,9.000000,5
49,6.9,0.19,0.35,5.0,0.067,32.0,150.0,0.99500,3.36,0.48,9.800000,5
...,...,...,...,...,...,...,...,...,...,...,...,...
2372,6.0,0.33,0.18,3.0,0.036,5.0,85.0,0.99125,3.28,0.40,11.500000,4
2133,8.3,0.24,0.27,2.1,0.030,22.0,162.0,0.99140,2.99,0.68,11.900000,6
4802,5.8,0.28,0.34,2.2,0.037,24.0,125.0,0.98986,3.36,0.33,12.800000,8
4734,5.5,0.16,0.31,1.2,0.026,31.0,68.0,0.98980,3.33,0.44,11.633333,6


In [4]:
train_x=train.drop("quality",axis=1).values
train_y=train["quality"].values.ravel()

# test datset
test_x=test.drop("quality",axis=1).values
test_y=test["quality"].values.ravel()

## split this train data into train and validation
train_x,valid_x,train_y,valid_y=train_test_split(train_x,train_y,test_size=0.2,random_state=42)

signature=infer_signature(train_x,train_y)

In [5]:
def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):

    ## model architecture
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)

    model = keras.Sequential([
        keras.Input([train_x.shape[1]]),
        keras.layers.Normalization(mean=mean, variance=var),
        keras.layers.Dense(64, activation="relu"),
        keras.layers.Dense(1)
    ])

    ## compile the model
    model.compile(
        optimizer=keras.optimizers.SGD(
            learning_rate=params["lr"],
            momentum=params["momentum"]
        ),
        loss="mean_squared_error",
        metrics=[keras.metrics.RootMeanSquaredError()]
    )

    ## train the model with mlflow tracking
    with mlflow.start_run(nested=True):
        model.fit(train_x, train_y, validation_data=(valid_x, valid_y),
                  epochs=epochs, batch_size=32)

        ## evaluate the model
        eval_result = model.evaluate(test_x, test_y)  # Now test_x is passed correctly
        eval_rmse = eval_result[1]

        ## log the parameters and metrics
        mlflow.log_params(params)
        mlflow.log_metrics({"rmse": eval_rmse})

        ## log the model
        mlflow.tensorflow.log_model(model, "model", signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}


In [6]:
def objective(params):
    ## mlflow will track the parameters and results for each run
    result=train_model(
        params=params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y
    )
    return result

In [7]:
space={
    "lr":hp.loguniform("lr",np.log(0.00001),np.log(0.1)),
    "momentum":hp.uniform("momentum",0.1,0.9)
}

In [8]:
mlflow.set_experiment("Wine-quality")
with mlflow.start_run():
    ## conduct the hyperparameter search
    trials=Trials()
    best=fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    ## get the best parameters
    best_run=sorted(trials.results,key=lambda x:x["loss"])[0]

    ##log the best parameters,loss and model
    mlflow.log_params(best)
    mlflow.log_metrics({"rmse":best_run["loss"]})
    mlflow.tensorflow.log_model(best_run["model"],"model",signature=signature)

    ## print out the best parameters and corresponding loss
    print("Best parameters:",best)
    print("Best loss:",best_run["loss"])

2025/03/09 14:33:22 INFO mlflow.tracking.fluent: Experiment with name 'Wine-quality' does not exist. Creating a new experiment.


Epoch 1/3                                            

 1/98 ━━━━━━━━━━━━━━━━━━━━ 36s 375ms/step - loss: 39.8293 - root_mean_squared_error: 6.3110
42/98 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 40.9332 - root_mean_squared_error: 6.3978   
97/98 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 40.2339 - root_mean_squared_error: 6.3428
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 40.2102 - root_mean_squared_error: 6.3409 - val_loss: 37.1181 - val_root_mean_squared_error: 6.0925

Epoch 2/3                                            

 1/98 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 33.5193 - root_mean_squared_error: 5.7896
58/98 ━━━━━━━━━━━━━━━━━━━━ 0s 880us/step - loss: 36.1704 - root_mean_squared_error: 6.0141
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 35.8956 - root_mean_squared_error: 5.9911 - val_loss: 33.5639 - val_root_mean_squared_error: 5.7934

Epoch 3/3                                            

 1/98 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - loss: 33.2009 - root_mean_squared_error: 5.